## Импортирую файлы с сервисами, привожу их к единому виду для последующего объединения 

In [2]:
import geopandas as gpd

bakery = gpd.read_file("./data/bakery.geojson")
bakery["id"] = range(len(bakery))
bakery["municipality_id"] = 0    # задаю недостающее для работы библиотеки поле
bakery["service_code"] = 'bakery'   # задаю недостающее для работы библиотеки поле
bakery["administrative_unit_id"] = 0   # задаю недостающее для работы библиотеки поле

In [3]:
blocks = gpd.read_file("./data/blocks.geojson")
blocks["block_id"] = range(len(blocks))

In [4]:
kindergarten = gpd.read_file("./data/kindergarten.geojson")
kindergarten["id"] = range(len(kindergarten))
kindergarten["service_code"] = 'kindergarten'   # задаю недостающее для работы библиотеки поле
kindergarten["municipality_id"] = 0   # задаю недостающее для работы библиотеки поле
kindergarten["administrative_unit_id"] = 0   # задаю недостающее для работы библиотеки поле

In [5]:
supermarket = gpd.read_file("./data/supermarket.geojson")
supermarket["id"] = range(len(supermarket))
supermarket["service_code"] = 'supermarket'   # задаю недостающее для работы библиотеки поле
supermarket["municipality_id"] = 0       # задаю недостающее для работы библиотеки поле
supermarket["administrative_unit_id"] = 0    # задаю недостающее для работы библиотеки поле
supermarket = supermarket.drop(columns=['bbox_north_1', 
       'bbox_south_1', 'bbox_east_1', 'bbox_west_1', 'place_id_1',
       'osm_type_1', 'osm_id_1', 'lat_1', 'lon_1', 'display_name_1', 'class_1',
       'type_1', 'importance_1', 'bbox_north_2', 'bbox_south_2', 'bbox_east_2',
       'bbox_west_2', 'place_id_2', 'osm_type_2', 'osm_id_2', 'lat_2', 'lon_2',
       'display_name_2', 'class_2', 'type_2', 'importance_2'])                   # задаю недостающее для работы библиотеки поле

## Объединяю все сервисы в единый датафрейм

In [6]:
bak = bakery.append(kindergarten).append(supermarket).sjoin(blocks, how="left")
bak = bak.drop(columns=["index_right"])
bak["block_id"] = bak["block_id"].fillna(bak["block_id"].median())
bak = bak.drop(columns=['Name','address', 'latitude', 'longitude'])
bak = bak.astype({"block_id": int})

C:\Users\danil\AppData\Local\Temp\ipykernel_1996\2178411597.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bak = bakery.append(kindergarten).append(supermarket).sjoin(blocks, how="left")
C:\Users\danil\AppData\Local\Temp\ipykernel_1996\2178411597.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bak = bakery.append(kindergarten).append(supermarket).sjoin(blocks, how="left")


In [7]:
bak.to_file("./data/final_services.geojson", driver="GeoJSON")

## Создаю модель города с помощью библиотеки

In [1]:
from CityGeoTools.metrics.data import CityInformationModel as BaseModel
city_model = BaseModel.CityInformationModel(city_name="Kaliningrad", city_crs=32634)
city_model.update_layer(attr_name="Services", file_name="./data/final_services.geojson")   # добавляю слой с сервисами
city_model.methods.services_clusterization.message   
# сообщение о том, прошла ли кластеризация успешно

c:\users\alexk\appdata\local\programs\python\python38\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Validation of Services layer...
Services layer loaded successfully!


{'Services': 'Layer matches specification'}

## Кластеризация сервисов

In [4]:
from CityGeoTools.metrics.calculations import services_clusterization 

service_clusteriztion = services_clusterization.ServicesClusterization(city_model).get_clusters_polygon(
    service_types=['bakery','kindergarten', 'supermarket'], n_std=2, condition_value=1000
    )

service_poins = gpd.GeoDataFrame.from_features(service_clusteriztion['services']).set_crs(4326)
service_clusters = gpd.GeoDataFrame.from_features(service_clusteriztion['polygons']).set_crs(4326)   # получаю геодатафрейм с кластерами

service_clusters.head(10)

,geometry,bakery,kindergarten,supermarket
0,"POLYGON ((20.35254 54.72409, 20.35255 54.72404...",0.00,1.00,0.00
1,"POLYGON ((20.43055 54.71378, 20.43054 54.71374...",0.00,1.00,0.00
2,"POLYGON ((20.44327 54.70979, 20.44187 54.70974...",0.00,0.75,0.25
3,"POLYGON ((20.45079 54.71835, 20.44969 54.71783...",0.25,0.50,0.25
4,"POLYGON ((20.46299 54.71493, 20.46202 54.71524...",0.00,1.00,0.00
5,"POLYGON ((20.45523 54.71448, 20.45377 54.71476...",0.00,1.00,0.00
6,"POLYGON ((20.46071 54.70913, 20.45921 54.70890...",0.67,0.33,0.00
7,"POLYGON ((20.48942 54.71515, 20.48821 54.71475...",0.17,0.67,0.17
8,"POLYGON ((20.45991 54.72262, 20.45883 54.72318...",0.40,0.40,0.20
9,"POLYGON ((20.47390 54.72559, 20.47311 54.72514...",1.00,0.00,0.00


In [5]:
service_poins      # геодатафрейм с сервисами и присвоенными номерами кластеров

,geometry,id,municipality_id,service_code,administrative_unit_id,block_id,cluster
0,POINT (20.51262 54.71719),0,0,bakery,0,597,79
1,POINT (20.50639 54.71220),1,0,bakery,0,499,81
2,POINT (20.50581 54.70257),2,0,bakery,0,509,65
3,POINT (20.50300 54.71476),3,0,bakery,0,523,80
4,POINT (20.50269 54.70937),4,0,bakery,0,421,82
...,...,...,...,...,...,...,...
348,POINT (20.55920 54.68208),148,0,kindergarten,0,845,96
349,POINT (20.33722 54.64952),157,0,kindergarten,0,0,97
350,POINT (20.41535 54.78807),158,0,kindergarten,0,499,98
351,POINT (20.62380 54.67359),169,0,kindergarten,0,915,99


In [11]:
service_clusters.to_file("clusters_services.geojson", driver="GeoJSON")

## Создаю карту с кластерами

In [12]:
!pip install folium matplotlib mapclassify
import folium
import osmnx
import geopandas
m = service_poins.explore(height=700, width=700, name="Polygons")
m = service_clusters.explore(m=m, color="red", name="Points")
folium.LayerControl().add_to(m)
m

You should consider upgrading via the 'c:\users\danil\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


На данной карте отображены кластеры, которые в себя включают имеющиеся сервисы. Данные кластеры представляют собой выпуклую оболочку, построенную вокруг указанных сервисов. В характеристиках кластера указано процентное соотношение сервисов внутри кластера. Значения меняются в диапазоне [0, 1]